In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [288]:
# GDP_NOM_SECTORS: Nominal GDP of different sectors
# GDP_YOY_SECTORS: Year-on-year real GDP growth rates for different sectors
# AR: Agriculture, Forestry and Fisheries
# ID: Industrial
# CO: Construction
# WH: Wholesale and retail trade
# TR: Transportation, storage and postal services
# AC: Accommodation and catering
# FI: Finance
# RE: Real Estate
# IN: Information transmission, software and information technology services
# LE: Leasing and business services
# OT: Other Industries

In [290]:
GDP_NOM_SECTORS = pd.read_csv('GDP_NOM_SECTORS.csv').set_index('QTR')
GDP_YOY_SECTORS = pd.read_csv('GDP_YOY_SECTORS.csv').set_index('QTR')
GDP_YOY_SECTORS['OT'] = (GDP_NOM_SECTORS[['IN', 'LE', 'OT']].sum(axis = 1) / np.multiply(GDP_NOM_SECTORS[['IN', 'LE', 'OT']] , 1 / (1 + GDP_YOY_SECTORS[['IN', 'LE', 'OT']]/ 100)).sum(axis = 1) - 1 ) * 100
GDP_YOY_SECTORS = GDP_YOY_SECTORS.drop(columns=['IN','LE'])
GDP_NOM_SECTORS['OT'] = GDP_NOM_SECTORS[['IN', 'LE', 'OT']].sum(axis = 1)
GDP_NOM_SECTORS = GDP_NOM_SECTORS.drop(columns=['IN','LE'])
GDP_YOY_SECTORS

,AR,ID,CO,WH,TR,AC,FI,RE,OT
QTR,,,,,,,,,
1992-03,8.9,18.3,18.1,11.7,7.8,24.2,4.0,18.7,8.000000
1992-06,6.1,20.0,20.6,10.2,9.4,25.5,5.9,21.6,12.700000
1992-09,3.4,21.0,22.1,11.8,16.0,29.6,8.2,27.9,14.400000
1992-12,3.5,24.8,21.6,8.8,8.3,28.5,7.8,34.7,11.800000
1993-03,5.0,20.8,22.9,9.5,10.7,11.0,10.1,17.4,13.600000
...,...,...,...,...,...,...,...,...,...
2021-12,6.4,3.8,-2.1,5.9,4.0,4.7,5.5,-2.9,6.085922
2022-03,6.1,6.4,1.4,3.9,2.1,-0.3,5.1,-2.0,5.928831
2022-06,4.5,0.4,3.6,-1.8,-3.5,-5.3,5.9,-7.0,0.814498


In [291]:
from rpy2.robjects import r
from rpy2.robjects.packages import importr
from rpy2.robjects import globalenv
from rpy2.robjects import pandas2ri
pandas2ri.activate()

BVAR = importr("BVAR")

Unable to determine R home: [WinError 2] The system cannot find the file specified


In [292]:
rdf = pandas2ri.py2rpy(GDP_YOY_SECTORS)
globalenv['rdf'] = rdf
rscript = """
bv_priors <- bv_mn( 
lambda = bv_lambda(mode = 0.6 ),
alpha = bv_alpha(mode = 0.4), 
var = 10000000, 
b = 1
) 
x <- bvar(rdf, lags = 6, n_draw = 1000L, n_burn = 200L, priors = bv_priors(), verbose = FALSE)
predict(x) <- predict(x, horizon = 2)
irf(x) <- irf(x, horizon = 2, fevd = FALSE)
summary(x)
"""
result = r(rscript)

In [296]:
result[0][10][0]

array([[[ -2.29972379, -12.64096349, -19.48138793, ...,  -6.2347984 ,
          -5.24612874,  -3.39173275],
        [  6.82032535,  14.37198473,  14.06689608, ...,   0.39688588,
          -6.40971595,  10.30085941]],

       [[  9.36719282,  14.97401708,  22.45074623, ...,  11.06796227,
           9.95886744,  11.09425726],
        [  8.22424569,  13.83361079,  21.41113302, ...,  19.59081987,
           0.77156972,   9.33033553]],

       [[  5.58083752,   6.47980861,  17.693657  , ...,  11.07292341,
         -15.17184094,   2.03813193],
        [  6.51324003,   8.53972137,  11.93900369, ...,   4.54113922,
           4.78030859,   1.11780663]],

       ...,

       [[  3.44180168,   0.58975253,   7.00325453, ...,  23.44345774,
           1.64368348,   8.64140355],
        [ 17.08486627,  56.58327736,  70.80045493, ...,  26.86339887,
          66.67125012,  23.42127159]],

       [[ -0.67398588, -10.11107084, -11.69156846, ...,  18.50408157,
         -17.73202827,   6.57954747],
       

186812.68421052632